In [ ]:
# To get the best performance out of LlamaCPP -  install the package so that it is compilied with GPU support.
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 79.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 197.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.63-cp310-cp310-linux_x86_64.whl size=39202691 sha256=a2c46a02fe0674f3b68495a939450cea0435f30642e1349fa8d020047193c302
  Stored in directory: /tmp/pip-ephem-wheel-cache-d5bi4jgw/wheels/5c/32/58/9b1aa714aea4ce0f0e0ee441f40d35c869aa6d40296437c796
Successfully built llama-cpp-python


In [ ]:
!pip install -q PyPDF2
!pip install -q python-dotenv
!pip install -q llama-index
!pip install -q transformers
!pip install -q llama-index-llms-llama-cpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

In [ ]:
model_url = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf"

In [ ]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

total size (MB): 4368.44


4167it [00:34, 120.47it/s]                          
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u3

In [ ]:
# upload pdf documents
!mkdir /content/data
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  !cp "{fn}" "/content/data/{fn}"

In [ ]:
import PyPDF2
def extract_text_from_pdf(pdf_file):
    # Use PyPDF2 to extract text from PDF
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

In [ ]:
import os
# get all chunks from pdfs
text_chunks = []
for filename in os.listdir("/content/data/"):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join("/content/data/", filename)
        pdf_text = extract_text_from_pdf(pdf_path)
        text_chunks.extend(chunk_text(pdf_text))

In [ ]:
def limit_mcq_options(mcq_json):
    """Limits MCQ options to four, ensuring at least one distractor."""
    mcq_json["options"] = random.sample(mcq_json["options"], k=4)  # Shuffle and take 4
    if mcq_json["answer"] not in mcq_json["options"]:
        mcq_json["options"][0] = mcq_json["answer"]
    return mcq_json

In [ ]:
import json
import random

def gnerate_questions(text_chunks, llm):

  results = {
        "mcq_questions": [],
        "true_false_questions": []
  }

  used_chunks = set()
  generated_questions = set()

  # Generate 5 sets of questions
  for _ in range(5):
    # Stop if no unique chunks left
    if len(used_chunks) == len(text_chunks):
            break

    chunk = random.choice(list(set(text_chunks) - used_chunks))
    used_chunks.add(chunk)

    prompt_mcq = f"""Generate a JSON-formatted response with a multiple-choice question (MCQ) based on this passage. Include the following:
                 * **question:** The question text (ask about a specific fact or ask to identify the INCORRECT statement).
                 * **options:** An array of four possible answer choices. Include ONE correct answer and three plausible distractors.
                 * **answer:** The single correct answer (marked within the 'options' array).
                 Passage: {chunk}"""

    prompt_true_false = f"""Generate a JSON-formatted response with a true/false statement based on this passage. Include the following:
                        * **statement:** A statement that is clearly true or false.
                        * **answer:** 'True' or 'False'.
                        Passage: {chunk} """


    # MCQ
    response_mcq = llm.complete(prompt_mcq)
    response_mcq_dict = json.loads(response_mcq.json())
    json_mcq_text = response_mcq_dict["text"]
    mcq_json = json.loads(json_mcq_text)

    mcq_json = limit_mcq_options(mcq_json)

    # duplication Check
    if mcq_json["question"] not in generated_questions:
      generated_questions.add(mcq_json["question"])
      results["mcq_questions"].append(mcq_json)

    # True/False
    response_tf = llm.complete(prompt_true_false)
    response_tf_dict = json.loads(response_tf.json())
    json_tf_text = response_tf_dict["text"]
    tf_json = json.loads(json_tf_text)

    # duplication Check
    if tf_json["statement"] not in generated_questions:
      generated_questions.add(tf_json["statement"])
      results["true_false_questions"].append(tf_json)

  return results

In [ ]:
gnerated_questions_json = gnerate_questions(text_chunks, llm)